# Learning goal 2: creating an interactive visualisation

## Background

I have worked with visualisations in the past of course, but never any interactive ones in Python. At the start of the group project, I intented to create an interactive visualisation showing the output of the urban wind models we wanted to test. Since we did not find the correct models, I instead created a visualisation of the Netatmo data.

## Data source

For simplicity's sake, I have used the same data set as in the first learning goal. It is a much smaller file to download compared to the full dataset, while it does not impede the quality of the visualisation. The only downside is that the time range able to visualise is a lot smaller. 

The information that is used from the dataset this time are the date and time, coordinates, wind direction, and wind speed.

## Methodology & Implementation

Firstly, the necessary packages are imported. In order to be able to run the interactive visualisation, it is necessary to install missing packages on your computer, as well as install an extension that enables widgets in Jupyter Notebook. Run the following in your terminal (or Anaconda Prompt):

 `conda install -c conda-forge ipywidgets` <br/>
 `pip install ipywidgets` <br/>
 `jupyter nbextension enable --py widgetsnbextension`

Some parts of the visualisation do not work in Safari. It is best to use Chrome, Firefox, or Edge.

In [1]:
import folium
import pandas as pd
from folium import plugins
from folium.plugins import BoatMarker
import matplotlib
import branca
import ipywidgets as widgets
from IPython.display import display
from datetime import datetime

netatmo = pd.read_csv('PWS_first100k_withPressureTrends.csv')

Next, two functions are defined that are later used in the visualisation, `findColorValue` and `createWindMapAmsterdam`. The first function finds the color hue for a particular wind speed. Wind speeds can range from 1 km/h to 38 km/h, with the colors going from light yellow to dark red. The function receives a predefined color scale called 'colormap' and a wind speed, looks up the correct hue, converts it from RGB to a hexadecimal color code and returns the code.

The second function, `createWindMapAmsterdam`, creates a Folium map and populates it with observations from the Netatmo dataset. It takes a certain time as input argument, and subsets all observations from that time in the Netatmo dataset. Any observations with 'NA' wind speed measurements are deleted. Then, the color scale and legend are created. At last, each observation from that timestamp is added to the Folium map, where the wind direction of the observation is reflected in the orientation of the marker, and the color of the marker indicates the wind speed.

In [2]:
def findColorValue(colormap, wind_speed):
    rgb = colormap.rgba_floats_tuple(wind_speed)[:-1]
    return(matplotlib.colors.to_hex(rgb))

In [3]:
def createWindMapAmsterdam(desired_timestamp):
    netatmo_timestamp = netatmo.loc[netatmo['timestamp'] == desired_timestamp]
    netatmo_timestamp = netatmo_timestamp[netatmo_timestamp['wind_speed_kmh'].notna()]
    
    map = folium.Map([52.37, 4.9], zoom_start=11)

    colormap = branca.colormap.linear.YlOrRd_09.scale(1, 38)
    colormap = colormap.to_step(index=[1, 3, 5, 8, 11, 15, 19, 23, 28, 33, 38])
    colormap.caption = 'Measured wind speed (km/h)'
    colormap.add_to(map)

    for i, (lat, lon) in enumerate(zip(netatmo_timestamp.loc[:, "y"], netatmo_timestamp.loc[:, "x"])):
        folium.plugins.BoatMarker(
            location=[lat,lon],
            wind_speed=netatmo_timestamp.iloc[i]["wind_speed_kmh"],
            heading=int(netatmo_timestamp.iloc[i]["wind_angle"]),
            color=findColorValue(colormap, int(netatmo_timestamp.iloc[i]["wind_speed_kmh"]))
        ).add_to(map)
    return(map)
    
map2 = createWindMapAmsterdam("2017-07-19 18:00:00")

Next, the previous two functions are used in the function called `mapPlotter`. This function is interactive and is called every time the user changes the input of the two user interface elements. The function appends the date and time that the user inputs into one string and calls the `createWindMapAmsterdam` function. The map is thus redrawn each time the user inputs new values.

The other code in the code block below initializes and handles the UI-elements.

In [4]:
datepicker = widgets.DatePicker(
    description='Choose date:',
    disabled=False,
    value=datetime.strptime("2017-06-01", '%Y-%m-%d')
)

timepicker = widgets.IntSlider(
    min = 0,
    max = 23,
    step = 1,
    description = "Choose hour:",
    value=19
)

widgetMapOutput = widgets.Output()

def mapPlotter(datepicker, timepicker):
    widgetMapOutput.clear_output()
    with widgetMapOutput:
        if len(str(timepicker)) == 1:
            time = " 0" + str(timepicker) + ":00:00"
        elif len(str(timepicker)) == 2:
            time = " " + str(timepicker) + ":00:00"
        datetime = str(datepicker) + time
        print("Measurements at " + datetime)
        WindMap = createWindMapAmsterdam(datetime)
        display(WindMap)
    
def datepicker_eventhandler(change):
    mapPlotter(change.new, timepicker.value)
    
def timepicker_eventhandler(change):
    mapPlotter(datepicker.value, change.new)

## Results

The code below simply shows the UI-elements and their output, and makes them responsive.

In [5]:


datepicker.observe(datepicker_eventhandler, names="value")
timepicker.observe(timepicker_eventhandler, names="value")

display(datepicker)
display(timepicker)
display(widgetMapOutput)


DatePicker(value=datetime.datetime(2017, 6, 1, 0, 0), description='Choose date:')

IntSlider(value=19, description='Choose hour:', max=23)

Output()


Change the initial date for the Folium map to show! The observations run from 2017-06-01 to 2017-08-05. The legend contains two colors for each wind speed category on the Beaufort scale. A new category begins each time a km/h value is shown, i.e. 1->5 = category 1, 5->11 = category 2, ...

## Conclusion



A few conclusions can be drawn when viewing the dataset through this visualisation. For some hours and days, there are very few observations available. This is because all wind measurements of below 1 km/h are removed from the dataset, as such low wind measurements are unreliable (Droste et al. 2020). Also, the measured wind direction varies greatly, even at times when there is lots of wind. For example, on 7-6-2017 at 06:00h, the reported wind speed at Schiphol airport was 12.0 m/s = 43.2 km/h. Some observations do show increased wind speeds, but many still report quite low observations coming from all kinds of directions. The wind direction measurements do seem more accurate as wind speeds increase. The stations that report the highest wind speeds consistently indicate correct wind directions. 

All in all, I learned quite a few things making this visualisation. Before, I did not know how to create interactive UI-elements in Python, and use these as inputs for a map. I had also never worked with the folium package. The data is now presented in a much more easily interpretable manner compared to a spreadsheet.


















